# 13장 모델의 성능 검증하기

1986년 제프리 힌튼 교수가 오차 역전파를 발표한 직후, 존스 홉킨스의 세즈노프스키(Sejnowski) 교수는 오차 역전파가 은닉층의 가중치를 실제로 업데이트시키는 것을 확인하고 싶었습니다. 그는 **광석과 일방 암석에 수중 음파 탐지기를 쏜 후 결과를 모아 데이터셋을 준비했고 음파 탐지기의 수신 결과만 보고 광석인지 일반 암석인지를 구부하는 모델을 만들었습니다.** 그가 측정한 결과의 정확도는 얼마였을까요?

<br><center>
<img src="https://drive.google.com/uc?id=1G-cPV2ET-IrDbdzQFh430UCMz7CHuPpL" width=400>
</center><br>


이 장에서는 세즈노프스키 교수가 했던 초음파 광물 예측 실험을 텐서플로로 재현해보고 이렇게 구해진 실험 정확도를 평가하는 방법과 성능을 향상시키는 중요한 머신 러닝 기법들에 대해 알아보겠습니다.

##  1. 데이터의 확인과 예측 실행

In [2]:
import pandas as pd

# 깃허브에 준비된 데이터를 가져옵니다.
!git clone https://github.com/taehojo/data.git

# 광물 데이터를 불러옵니다.
df = pd.read_csv('./data/sonar3.csv', header=None)

# 첫 5줄을 봅니다. 
df.head()

fatal: destination path 'data' already exists and is not an empty directory.


,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,0
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,0
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,0
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,0
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,0


첫번째 열(0)부터 60번째(59)열까지는 음파의 에너지를 0에서 1 사이의 숫자로 표시하고 있습니다. 이제 일반 암서과 광석이 각각 몇 개나 데이터셋에 포함되어 있는지 확인해보겠습니다.

In [ ]:
# 일반 암석(0)과 광석(1)이 몇 개 있는지 확인합니다.
df[60].value_counts()

1    111
0     97
Name: 60, dtype: int64

광석 샘플이 111개 암석 샘플이 97개 따라서 샘플 수는 총 111+97= 208개의 샘플이 데이터 셋을 구성하고 있습니다. 

1\~60번째(0~59) 열을 변수 $X$에 저장하고 광물의 종류를 $y$로 표현하겠습니다.

In [ ]:
# 음파 관련 속성을 X로, 광물의 종류를 y로 저장합니다.
X = df.iloc[:,0:60]
y = df.iloc[:,60]

이후 앞서 했던 그대로 딥러닝을 실행하겠습니다. 출력 $y$는 하나이며 은닉층...

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 모델을 설정합니다.
model = Sequential()
model.add(Dense(24,  input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델을 실행합니다.
history=model.fit(X, y, epochs=200, batch_size=10)

Epoch 1/200
21/21 [==============================] - 1s 2ms/step - loss: 0.6947 - accuracy: 0.4615
Epoch 2/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6833 - accuracy: 0.5385
Epoch 3/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6730 - accuracy: 0.5625
Epoch 4/200
21/21 [==============================] - 0s 3ms/step - loss: 0.6613 - accuracy: 0.6106
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6517 - accuracy: 0.6298
Epoch 6/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6391 - accuracy: 0.6587
Epoch 7/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6267 - accuracy: 0.6779
Epoch 8/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6070 - accuracy: 0.7067
Epoch 9/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5883 - accuracy: 0.7067
Epoch 10/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5643 - accuracy: 0.7644
Epoch 11/

200번 반복되었을 때의 결과를 보니 정확도가 100%입니다. 이 모델의 예측 정확도가 100%라는 것을 믿을 수 있습니까? 정말로 광석인지 일반 암석인지  100%의 확율로 판별해 내는 모델이 만들어진 것이라요? 다음 섹션에서 이 의문에 대한 답을 찾아 보겠습니다. 

## 2. 과적합 이해하기
이제 과적합 문제가 무엇인지 알아보고 이를 어떻게 해결하는지 살펴보겠습니다. 과적합(overfitting)이란 모델이 학습 데이터셋 안에서는 일정 수준 이상의 예측 정보를 보이지만 새로운 데이터에 적용하면 잘 맞지 않는 것을 의미합니다. 

그림(13-1)의 그래프에는 두 종류의 데이터가 있습니다. 원 안이 검은 색인 것과 원 안이 흰색인 두 종류의 데이터가 보입니다. 이 두 종류를 완벽하게 또는 매우 정확하게 분류하기 위해 구분 선을 심한 곡선으로 그린 것(과적합)과 그 외 두 가지의 직선 행태의 구분선이 보입니다.   
<br><center>
(그림. 13-1) 과적합<br>
<img src="https://drive.google.com/uc?id=1bPEOjpG1MqG1eF_WSZjHwDUpczsAnAU2" width=500>
</center><br>  

과적합 구분 선은 주어진 샘플들에만 최적화 되어 있습니다. 새로운 데이터가 주어졌을 때 과적합된, 그러니까 기존 데이터에 overfitting된 구분선으로 새로운 데이터를 정확히 불류하기 어렵다는 것입니다. 

과적합(overfitting)은 층이 너무 많거나 변수가 복자해서 살생하기도 하고 테스트셋과 학습셋이 중복될 때 생기기도 합니다. 특히 딥러닝은 학습 단계에서 입력층, 은닉층, 출력층의 노드들에 상당히 많은 변수가 투입됩니다. 따라서 딥러닝을 진행하는 동안 과적합에 빠지 않게 늘 주의해야 합니다. 


## 3. 학습셋과 테스트셋
그렇다면 과적합을 방지하려면 어떻게 해야할까요? 먼저 학습을 하는 데이터셋과 이를 테스트할 데이터셋을 완전히 구분한 후 학습과 동시에 테스트를 변행하며 진행하는 것이 한 방법입니다. 예를 들어 데이터세이 총 100개의 샘플로 이루어져 있다면 다음과 같이 두 개의 셋으로 나눕니다. 

<br><center>
(그림. 13-2) 학습셋과 테스트셋 구분<br>
<img src="https://drive.google.com/uc?id=1MBgSLj0CodjyAHGZx4o4EIvKDFLolpM1">
</center><br>

(전체 데이터셋의 70\~75%를 학습셋으로 사용하고 30\~25%를 테스트 셋으로 나눕니다.) 신경망을 만들어 70개의 샘플로 학습을 진행한 후 이 학습의 결과를 저장합니다. 이렇게 저당된 파일을 '모델'이라고 합니다. 모델은 다른 셋에 적용할 경우 학습 단계에서 각인되었던 그대로 다시 수행합니다. 따라서 나머지 30개의 샘플로 테스트해서 정확도를 살펴보면 학습이 얼마나 잘 되었는지 알 수 있을 것입니다. 딥러닝 같은 알고리즘을 충분히 조절해 가장 나은 모델이 만들어지면 이를 실생활에 대입해 활용하는 것이 바로 머신 러닝의 개발 순서입니다.  

<br><center>
(그림. 13-3) 학습셋과 테스트셋<br>
<img src="https://drive.google.com/uc?id=1BOHkL4fJVuVrCdIpfC9P02t-TUB8_f92" width=400>
</center><br>  
지금까지 우리는 테스트셋을 만들지 않고 모든 데이터셋을 이용해 학습시켰습니다. 그런데로 매번 정확도(accuracy)를 계산할 수 있었지요. 어떻게 가능했을까요?  

**"지금까지 학습데이터를 이용해 정확도를 측정한 것은 데이터셋에 들어있는 모든 샘플을 그대로 테스트에 활용한 결과입니다." → 지금까지 모든 데이터셋을 이용해 학습했고 모든 데이터셋(데이터 샘플)을 가지고 테스트한 결과입니다.** 

이를 통해 학습이 진행되는 상황을 파악할 수는 있지만 새로운 데이터에 적용했을 때 어느 정도의 성능이 나올지 알수 없습니다. 머신 러닝의 최종 목적은 과거의 데이터를 토대로 새로운 데이터를 예측하는 것입니다. 즉, 새로운 데이터에 사용할 모델을 만드는 것이 최종 목적이므로 테스트셋을 만들어 정확한 평가를 병행하는 것이 매우 중요합니다. 

학습셋만 가지고 평가할 때, 층을 더하거나 에포크(epoch) 값을 높여 실행 횟수를르리면 정확도가 계소해서 올라갈 수 있습니다. 하지만 학습 데이터셋만으로 평가한 예측 성공률이 테스트셋에서도 그래도 나나타지는 않습니다. 즉, 학습이 깊어져 학습셋 내부에서 성공률은 높아져도 테스트셋에서는 효과가 없다면 과적합이 일어난 것이지요. 이를 그래프로 표현하면 그림(13-4)와 같습니다.

<br><center>
(그림. 13-4) 학습이 계속되면 학습셋에서는 에러는 계속해서 작아져 과적합 발생<br>
<img src="https://drive.google.com/uc?id=1ZrnHjrDC6TXjPww9R5JFyYrwHrDyNIME" width=400>
</center><br>  

학습을 진행해도 테스트 결과가 더 이상 좋아지지 않는 시점에서 학습을 멈춰야 합니다. 이때 학습 정도가 가장 적절한 것으로 볼 수 있습니다. 

우리가 다루는 초음파 광물 예측 모델을 만든 세즈노프스키 교수가 실험 결과를 발표한 논무의 일부를 가져와 보겠습니다. 

<br><center>
(그림. 13-5) 학습셋과 테스트셋 정확도 측정의 예(RP Gorman et.al., 1998)<br>
<img src="https://drive.google.com/uc?id=1pnJsvvIZ_RJwDhyQ63nGsgOIKofvPv9L">
</center><br>  

여기서 눈여겨보아야 할 부분은 은닉층(Number of Hidden Units) 개수가 올라감에 따라 학습셋의 예측율(Average Performacne on Training Sets)과 데스트셋의 예측률(Average Performance on Testing Sets)이 어떻게 변하는지입니다. 이 부분만 따로  뽑아서 정리하면 표(13-2)와 같습니다. 

<br><center>
(표. 13-2) 은닉층 개수의 변화에 따른 학습셋의 예측률<br>

|은닉층 개수|학습셋의 예측률|테스트셋의 예측율|
|:---:|---:|---:|
|0|79.3|73.1|
|2|96.2|85.7|
|3|98.1|87.6|
|6|99.4|89.3|
|12|99.8|90.4|
|24|100|89.2|

</center><br>  

은닉층이 늘어날수록 학습셋의 에측률이 점점 올라가다가 결국 24개 층에 이르면 100% 예측률을 보입니다. 우리가 조금 전에 실행했던 결과와 같습니다. 그런데 이 모델을 토대로 테스트한 결과는 어떤가요? 테스트셋 예측률은 은닉층의 개수가 12개일 때 90.4%로 최고를 이루다 24개째에서는 다시 89.2%로 떨어지고 맙니다. 즉, 식이 복잡해지고 학습량이 늘어날수록 학습 데이터를 통한 예측률은 계속해서 올라가지만 은닉층의 수를 적절하게 조절하지 않을 경우 테스트셋을 이용한 예측률은 오히려 떨어지는 것을 확인할 수 있습니다. 

그러면 예제에 주어진 데이터를 학습 데이터셋과 테스트셋으로 나누는 예제를 만들어 보겠습니다. 

🚀 여기서 잠깐.  
이 실습에서는 사이킷런(scikit-learn) 라이브러리가 필요합니다.

저장된 $X$ 데이터와 $y$ 데이터에서 각각 정해진 비율(%)만큼 학습 데이터셋과 테스트 데이터셋으로 분리시키는 함수가 사이킷런의 ```train_test_split()```함수입니다. 따라서 다음과 같이 학습 데이터셋과 테스트 데이터셋을 만들 수 있습니다. 총 데이터셋에서 학습 데이터셋을 70%, 테스트 데이터셋을 30%로 나눌 때의 코드  
예입니다. 
```
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, shuffle=True)
```
위 코드에서 ```test_size```은 테스트 데이터셋의 비율입니다. 0.3은 전체 데이터셋의 30%를 테스트 데이터셋으로 사용하겠다는 것으로 나머지 70%를 학습 데이터셋으로 사용하게 됩니다. 이렇게 나누어진 학습 데이터셋과 테스트 데이터셋으로 각각 ```X_train, y_train``` 그리고 ```X_test, y_test```에 저장됩니다. 

모델은 앞서 만든 구조를 그대로 유지하고 모델에 성능 평가를 위해 테스트 함수(```model.evaluate()```)를 추가했습니다. 

```
score = model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])
```
```model.evaluate()``` 함수는 loss와 accuracy 두 가지를 계산해 출력합니다. 이를 score에 저장하고 accuracy를 출력하도록 했습니다. 

이제 전체 코드를 실행해 보겠습니다.




In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

import pandas as pd

In [ ]:
# 깃허브에 준비된 데이터를 가져옵니다. 앞에서 이미 데이터를 가져왔으므로 추석 처리합니다. 3번 예제만 별도 실행 시 주석을 해제하여 실습하세요.
!git clone https://github.com/taehojo/data.git

fatal: destination path 'data' already exists and is not an empty directory.


In [ ]:
# 광물 데이터를 불러옵니다.
df = pd.read_csv('./data/sonar3.csv', header=None)

In [ ]:
# 음파 관련 속성을 X로, 광물의 종류를 y로 저장합니다.
X = df.iloc[:,0:60]
y = df.iloc[:,60]

In [ ]:
# 학습셋과 테스트셋을 구분합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [ ]:
# 모델을 설정합니다.
model = Sequential()
model.add(Dense(24,  input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델을 실행합니다.
history=model.fit(X_train, y_train, epochs=200, batch_size=10)

Epoch 1/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6916 - accuracy: 0.5103
Epoch 2/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6763 - accuracy: 0.5724
Epoch 3/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6651 - accuracy: 0.5793
Epoch 4/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6553 - accuracy: 0.6345
Epoch 5/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6451 - accuracy: 0.6552
Epoch 6/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6348 - accuracy: 0.6828
Epoch 7/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6262 - accuracy: 0.6690
Epoch 8/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6142 - accuracy: 0.7172
Epoch 9/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6028 - accuracy: 0.6966
Epoch 10/200
15/15 [==============================] - 0s 2ms/step - loss: 0.5901 - accuracy: 0.7103
Epoch 11/

In [ ]:
# 모델을 테스트셋에 적용해 정확도를 구합니다. 
score=model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])

2/2 [==============================] - 0s 6ms/step - loss: 0.7985 - accuracy: 0.8254
Test accuracy: 0.8253968358039856


학습 데이터셋(X_train과 y_train)을 이용해 200번의 학습을 진행했을 때 모델이 판단한 학습 데이터셋에 대한 정확도와 생성된 모델을 테스트셋을 적용했을 때 보인 정확도가 다르다는 것입니다. 테스트 데이터셋을 적용했을 때의 정확도가 학습 데이터셋를 활용해 생성한 모델이 학습 데이터셋를 가지고 판단한 정확도 보다 낮습니다. 


머신러인, 딥러닝의 목표는 학습 데이터셋에서만 잘 동작하는 모델을 만드는 것이 아니라. 새로운 데이터에 대해 높은 정확도를 안정되게 보여주는 모델을 만드는 것이 목표입니다. 어떻게 하면 그러한 모델을 만들 수 있을까요? 모델 성능의 향상을 위한 방법에는 크게 데이터를 보강하는 방법과 알고리즘을 최적화 하는 방법이 있습니다. 

데이터를 이용해 성능을 향상시키려면 우선 충분한 데이터를 가져와 구가하면 됩니다. 많이 알려진 아래 그래프는 특히 딥러닝의 경우 샘플 수가 많을 수록 성능이 좋아짐을 보여줍니다.  

<br><center>
(그림. 13-6) 데이터의 증가와 딥러닝, 머신러닝 성능의 상관관계<br>
<img src="https://drive.google.com/uc?id=1WELFk7zpYiBYWmMnd7eJ7tlGXR36xrTL" width=400>
</center><br>  

하지만 데이터를 추가하는 것 자체가 어렵거나 데이터 추가만으로 성능 개선에 한계가 있을 수 있습니다. 딸서 가지고 있는 데이터를 적절히 보완해 주는 방법을 사용합니다. 예를 들어 사진의 경우 크기를 확대/축소한 것을 데이터 셋에 추가해 보거나 위 아래로 조금씩 움직인 사진을 데이터셋에 추가하는 것입니다.(이 내용은 20장에서 다룹니다) 테이블형 테이터의 경우 너무 크거나 낮은 이상치가 모델에 영향을 줄 수 없도록 크기를 적절히 조절할 수 있습니다. 시그모이드 함수를 사용해 전체를 0~1사이의 값으로 변환하는 것이 좋은 예입니다. 또 교차 검증 방법을 사용해서 가지고 있는 데이터를 충분히 이용하는 방법도 있습니다. 이는 잠시 후에 설명할 것입니다. 

다음으로 알고리즘을 이용해 성능을 향상하는 방법은 먼저 다른 구조로 모델을 바꾸어 가며 최적의 구조를 찾는 것입니다. 예를 들어 은닉층의 개수라든지, 그 안에 들어갈 노드의 수, 최적화 함수의 종류를 바꾸어 보는 것입니다. 앞서 이야기한 바 있지만 딥러닝 설정에 정답은 없습니다. 자신의 상황에 맞는 구조를 계속해서 테스트 해보며 찾는 것이 중요합니다. 그리고 데이터에 따라서는 딥러닝이 아닌 랜덤 포레스트, XGBoost, SVM 등 다른 알고리즘이 더 좋은 결과를 보일 때도 있습니다. 일반적인 머신 러닝과 딥러닝을 합해서 더 좋은 결과를 만드는 것도 가능하지요. 많은 경험을 통해 최적의 성능을 보이는 모델을 만드는 것이 중요합니다. 

**이제 현재 모델을 저장하고 불러오는 방법에 대해 알아보겠습니다.**

## 4. 모델 저장과 재사용
학습이 끝난 후 지금 만든 모델을 저장하면 언제든지 이를 불러와 다시 사용할 수 있습니다. 학습 결과를 저장하려면 ```model.save()```함수를 이용해 모델을 저장할 수 있습니다.

In [ ]:
# 모델 이름과 저장할 위치를 함께 지정합니다. 
model.save('./data/model/my_model.hdf5') 

hdf5 파일 포멧은 주로 과학 기술 데이터 작업에서 사용되는데, 크고 복잡한 데이터를 저장하는데 사용됩니다. 이를 다시 불러오려면 케라스 API의 ```load_model()```함수를 사용합니다. 앞서 ```Sequential()``` 함수를 불러온 모델 클래스 안에 함께 들어 있으므로 ```Sequential``` 뒤에 ```load_model```을 추가합니다.

In [ ]:
from tensorflow.keras.models import Sequential, load_model

좀전에 만든 모델을 메모리에서 삭제하겠습니다.

In [ ]:
# 테스트를 위해 조금 전 사용한 모델을 메모리에서 삭제합니다.
del model 

저장된 모델을 불러 옵니다.

In [ ]:
# 모델을 새로 불러옵니다.
model = load_model('./data/model/my_model.hdf5') 

# 불러온 모델을 테스트셋에 적용해 정확도를 구합니다. 
score=model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])

2/2 [==============================] - 0s 6ms/step - loss: 0.7985 - accuracy: 0.8254
Test accuracy: 0.8253968358039856


테스트 데이터셋을 가지고 정확도 검사를 다시 해봤습니다. 이전과 같은 결과를 얻은 것을 확인할 수 있습니다.

### [과제]
과제 1 - 데이터셋의 65%를 학습 데이터셋으로하고 35%를 데트트 데이터셋으로 나누어서 위 과정을 수행하십시요. 생성된 모델의 학습데이터셋에 대한 정확도와 테스트 데이터셋에 대한 정확도를 제시하십시요.

과제 2 - 데이터셋의 80%를 학습 데이터셋으로하고 20%를 데트트 데이터셋으로 나누어서 위 과정을 수행하십시요. 생성된 모델의 학습데이터셋에 대한 정확도와 테스트 데이터셋에 대한 정확도를 제시하십시요.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

import pandas as pd

# 광물 데이터를 불러옵니다.
df = pd.read_csv('./data/sonar3.csv', header=None)
# 음파 관련 속성을 X로, 광물의 종류를 y로 저장합니다.
X = df.iloc[:,0:60]
y = df.iloc[:,60]

# 학습셋과 테스트셋을 구분합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, shuffle=True)

# 모델을 설정합니다.
model = Sequential()
model.add(Dense(24,  input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델을 실행합니다.
history=model.fit(X_train, y_train, epochs=200, batch_size=10)

print('---'*24)
# 모델을 테스트셋에 적용해 정확도를 구합니다. 
score=model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])

Epoch 1/200
14/14 [==============================] - 0s 2ms/step - loss: 0.6834 - accuracy: 0.5556
Epoch 2/200
14/14 [==============================] - 0s 2ms/step - loss: 0.6739 - accuracy: 0.5852
Epoch 3/200
14/14 [==============================] - 0s 2ms/step - loss: 0.6674 - accuracy: 0.5704
Epoch 4/200
14/14 [==============================] - 0s 2ms/step - loss: 0.6606 - accuracy: 0.5926
Epoch 5/200
14/14 [==============================] - 0s 2ms/step - loss: 0.6556 - accuracy: 0.5926
Epoch 6/200
14/14 [==============================] - 0s 2ms/step - loss: 0.6462 - accuracy: 0.6074
Epoch 7/200
14/14 [==============================] - 0s 3ms/step - loss: 0.6403 - accuracy: 0.6074
Epoch 8/200
14/14 [==============================] - 0s 2ms/step - loss: 0.6340 - accuracy: 0.6370
Epoch 9/200
14/14 [==============================] - 0s 2ms/step - loss: 0.6247 - accuracy: 0.6444
Epoch 10/200
14/14 [==============================] - 0s 2ms/step - loss: 0.6176 - accuracy: 0.6444
Epoch 11/

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

import pandas as pd

# 광물 데이터를 불러옵니다.
df = pd.read_csv('./data/sonar3.csv', header=None)
# 음파 관련 속성을 X로, 광물의 종류를 y로 저장합니다.
X = df.iloc[:,0:60]
y = df.iloc[:,60]

# 학습셋과 테스트셋을 구분합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# 모델을 설정합니다.
model = Sequential()
model.add(Dense(24,  input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델을 실행합니다.
history=model.fit(X_train, y_train, epochs=200, batch_size=10)

print('---'*24)
# 모델을 테스트셋에 적용해 정확도를 구합니다. 
score=model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])

Epoch 1/200
17/17 [==============================] - 0s 2ms/step - loss: 0.7123 - accuracy: 0.4819
Epoch 2/200
17/17 [==============================] - 0s 2ms/step - loss: 0.6899 - accuracy: 0.5060
Epoch 3/200
17/17 [==============================] - 0s 2ms/step - loss: 0.6760 - accuracy: 0.5361
Epoch 4/200
17/17 [==============================] - 0s 2ms/step - loss: 0.6613 - accuracy: 0.5843
Epoch 5/200
17/17 [==============================] - 0s 2ms/step - loss: 0.6498 - accuracy: 0.6205
Epoch 6/200
17/17 [==============================] - 0s 2ms/step - loss: 0.6376 - accuracy: 0.6325
Epoch 7/200
17/17 [==============================] - 0s 2ms/step - loss: 0.6265 - accuracy: 0.7108
Epoch 8/200
17/17 [==============================] - 0s 2ms/step - loss: 0.6116 - accuracy: 0.7048
Epoch 9/200
17/17 [==============================] - 0s 2ms/step - loss: 0.5981 - accuracy: 0.7530
Epoch 10/200
17/17 [==============================] - 0s 2ms/step - loss: 0.5830 - accuracy: 0.7892
Epoch 11/

## 5. k겹 교차 검증

데이터가 충분히 많아야 모델 성능을 향상된다고 앞서 말했습니다. 이는 학습과 테스트를 위한 데이터를 충분히 확보할수록 세상에 나왔을 때 더 잘 동작하기 때문입니다. 하지만 실제 프로젝트에서는 데이터를 확보하는 것이 쉽지 않거나 많은 비용이 발생하는 경우도 있습니다. 따라서 가지고 있는 데이터를 십분 활용하는 것이 중요합니다. 특히 학습셋을 70%, 테스트셋을 30%로 설정할 경우 30%의 테스트셋은 학습에 이용할 수 없다는 단점이 있습니다.

이를 해결하기 위해 고안된 방법이 k겹 교차 검증(k-fold cross validation)입니다. k겹 교차 검증이란 먼저 데이터셋을 k 개로 나누고 그중 하나를 테스트셋으로 사용하고 테스트셋으로 선정하지 않은 나머지 데이터셋를 모두 합해서 학습셋으로 사용하여 정확도를 구합니다. 다시 k 개의 데이테셋에서 테스트셋으로 선택되지 않은 데이터셋을 테스트셋으로 사용하고 나머지 테이터셋을 모아 학습 데이터셋으로 사용하여 정확도를 구합니다. k개의 데이터셋을  다 한번씩 테스트 셋으로 두고 정확도를 구해서 얻은 k개의 정확도의 평균을 구해 최종 정확도를 판단합니다.

이렇게 하면 가지고 있는 데이터의 100%를 학습셋으로 사용할 수 있고 또 동시에 테스트셋으로도 사용할 수 있습니다. 예를 들어 5겹 교차 검증(5-fold cross validation)의 예가 그림(13-7)에 설며되어 있습니다.   
<br><center>
(그림. 13-7) 5겹 교차 검증 방법<br>
<img src="https://drive.google.com/uc?id=1UXPDWkDSPC0hhLCwxNwgWPuSfVPHw5Xm" width=500>
</center><br>  

이제 초음파 광물 예측 예제를 통해 5겹 교차 검증을 실히새 보겠습니다. 데이터를 원하는 수만큼 나우어 각각 학습셋과 테스트 셋으로 사용하게 하는 함수는 사이킷런 라이브러리의 ```KFold()```함수입니다. 실습 코드에서 ```KFold()```를 활용하는 부부만 뽑아 보며 다음과 같습니다. 

```
k=5
kfold = KFold(n_splits=k, shuffle=True)
acc_score=[]

for tranin_index, test_index in kfold.split(X):
   X_train, X_text = X.iloc[train_index, :], X.iloc[test_index,:]
   y_train, y_text = y.iloc[trian_index], y.iloc[test_index]
```

데이터셋을 몇개로 나눌 것인지 정해서 ```k```변수에 할당합니다. 사이킷런의 ```KFold()``` 함수를 불러 옵니다. ```shuffle```에 ```True```를 할당하면 데이터셋을 섞습니다. _k_번의 정확도 계산 결과(정확도 값)를 ```acc_score``` 리스트에 할당할 예정입니다. ```split()``` 함수에 의해 k개의 학습셋과 테스트 셋으로 분리되며 ```for``` 문에 의해 _k_번 반복됩니다. 

반복되는 매 학습 과정 마다 정확도를 구해 다음과같이 ```acc_score``` 리스트에 붙입니다.

```
accuracy = model.evaluate(X_test,y_test)   # 정확도를 구합니다.
acc_score.append(accuracy[1])              # acc_score 리스트에 저장합니다.
```

_k_번의 학습이 끝나면 각 정확도를 취합해 모델 성능 평가를 합니다. 아래에는 완성된 코드를 보입니다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

import pandas as pd

# 깃허브에 준비된 데이터를 가져옵니다. 앞에서 이미 데이터를 가져왔으므로 추석 처리합니다. 3번 예제만 별도 실행 시 주석을 해제하여 실습하세요.
# !git clone https://github.com/taehojo/data.git

# 광물 데이터를 불러옵니다.
df = pd.read_csv('./data/sonar3.csv', header=None)

# 음파 관련 속성을 X로, 광물의 종류를 y로 저장합니다.
X = df.iloc[:,0:60]
y = df.iloc[:,60]

In [ ]:
# 몇 겹으로 나눌 것인지를 정합니다. 
k=5

# KFold 함수를 불러옵니다. 분할하기 전에 샘플이 치우치지 않도록 섞어 줍니다.
kfold = KFold(n_splits=k, shuffle=True)

# 정확도가 채워질 빈 리스트를 준비합니다.
acc_score = []

# 모델 구조 생성
def model_fn():
    model = Sequential() # 딥러닝 모델의 구조를 시작합니다.
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

# K겹 교차 검증을 이용해 k번의 학습을 실행합니다. 
for train_index , test_index in kfold.split(X):  # for 문에 의해서 k번 반복합니다. spilt()에 의해 k개의 학습셋, 테스트셋으로 분리됩니다.
    X_train , X_test = X.iloc[train_index,:], X.iloc[test_index,:]  
    y_train , y_test = y.iloc[train_index], y.iloc[test_index]

    model = model_fn()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history=model.fit(X_train, y_train, epochs=200, batch_size=10, verbose=0) 
    
    accuracy = model.evaluate(X_test, y_test)[1]  # 정확도를 구합니다.
    acc_score.append(accuracy)  # 정확도 리스트에 저장합니다.

# k번 실시된 정확도의 평균을 구합니다.
avg_acc_score = sum(acc_score)/k

# 결과를 출력합니다.
print('정확도:', acc_score)
print('정확도 평균:', avg_acc_score)

2/2 [==============================] - 0s 7ms/step - loss: 0.4666 - accuracy: 0.8293
정확도: [0.7857142686843872, 0.8571428656578064, 0.8809523582458496, 0.7560975551605225, 0.8292682766914368]
정확도 평균: 0.8218350648880005


학습이 진행되는 과정을 화면에 출력되지 않게 하려고 ```model.fit()``` 함수의 파라메타 ```verbose```에 0을 할당했습니다.

🚀 잠깐만요.  
텐서플로 함수가 for문에 포함되는 경우 다음과 같은 WARNING 메시지가 나오는 경우가 있습니다. 텐서프로 구동에는 문제가 없으므로 그냥 진해하면 됩니다.  
WARNING:tensorflow: 5 out of the last 9 call to <function Model.make_test_function.<locals>.test_function at ....> triggered tf.function retracing...


<br>
이렇게 해서 가지고 있는 데이터를 모두 사용해 학습과 테스트를 진행했습니다. 이제 다음 장에서 학습 과정을 시각화해 보는 방법과 학습을 몇 번 반복할지(epochs) 스스로 판단하게 하는 방법 등을 알아보며 모델 성능을 향상시켜 보겠습니다. 